# Generate a COOLEST template file

In [16]:
import os

from coolest.template.lazy import *
from coolest.template import info
from coolest.template.standard import COOLEST
from coolest.template.json import JSONSerializer

from pprint import pprint

In [17]:
TEMPLATE_NAME = 'pemd_sersic'
TEMPLATE_DIR = '.'

In [18]:
# Setup cosmology
cosmology = Cosmology(H0=73.0, Om0=0.3)

# Create a couple of source galaxies at different redshifts
source_1 = Galaxy('a source galaxy', 2.0,
                    light_model=LightModel('Sersic'))

# Create a lens galaxy
lens_1 = Galaxy('a lens galaxy', 0.5,
                light_model=LightModel('Sersic'),
                mass_model=MassModel('PEMD'))

# Set some values to the lens mass parameters
lens_1.mass_model[0].parameters['theta_E'].set_point_estimate(1.27)
lens_1.mass_model[0].parameters['center_x'].set_point_estimate(0.1)
lens_1.mass_model[0].parameters['center_y'].set_point_estimate(-0.05)
lens_1.mass_model[0].parameters['gamma'].set_point_estimate(2.035)
lens_1.mass_model[0].parameters['phi'].set_point_estimate(32.)
lens_1.mass_model[0].parameters['q'].set_point_estimate(1.)

# Set some values to the source light parameters
source_1.light_model[0].parameters['theta_eff'].set_point_estimate(0.23)
source_1.light_model[0].parameters['center_x'].set_point_estimate(0.05)
source_1.light_model[0].parameters['center_y'].set_point_estimate(-0.1)
source_1.light_model[0].parameters['n'].set_point_estimate(2.3)
source_1.light_model[0].parameters['I_eff'].set_point_estimate(1.)
source_1.light_model[0].parameters['phi'].set_point_estimate(72.)
source_1.light_model[0].parameters['q'].set_point_estimate(1.)

# Put them in a list, which will also create unique IDs for each profile
entity_list = LensingEntityList(lens_1, source_1)

# Define the origin of the coordinates system
origin = CoordinatesOrigin()

# Provide data file
obs_pixels = PixelatedRegularGrid(field_of_view_x=(-3., 3.),
                                  field_of_view_y=(-3., 3.),
                                  num_pix_x=100, num_pix_y=100)

# Select the type of noise
from coolest.template.classes.noise import UniformGaussianNoise
noise = UniformGaussianNoise(std_dev=0.004)

observation = Observation(pixels=obs_pixels)

# Defines the instrument
# from coolest.template.classes.psf import GaussianPSF
# psf = GaussianPSF(0.2)

instrument = Instrument(0.06)


Group all instances above into a "master" COOLEST object

In [19]:
# Master object for the standard
coolest = COOLEST('MAP',
                    origin,
                    entity_list,
                    observation, 
                    instrument, 
                    cosmology)


Export it as the COOLEST template (JSON format)

In [20]:
# export as JSON file
template_path = os.path.join(os.getcwd(), TEMPLATE_DIR, TEMPLATE_NAME)
serializer = JSONSerializer(template_path, obj=coolest,
                            check_external_files=True)

# or using pure JSON (more human-readable)
serializer.dump_simple()